### Code above is what was used to retrieve the data in the "no_matching_player_with_sofifaid" csv. We will be using the csv to load faster

import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://sofifa.com/players"

no_match_df = pd.read_csv('../raw_data/not_matching_names.csv')
no_match_df['sofifa_id'] = 0

for idx, name in enumerate(no_match_df['name']):
    params = {
        'keyword': name
    }
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        sofifa_id = soup.find("img", class_="player-check").get('id')
        no_match_df['sofifa_id'][idx] = sofifa_id
    except AttributeError:
        no_match_df['sofifa_id'][idx] = 'Nan'

In [1]:
import pandas as pd

In [5]:
no_match_df=pd.read_csv("../raw_data/no_matching_player_with_sofifaid")

In [6]:
no_match_df

,Unnamed: 0,name,sofifa_id
0,0,Ángel Di María,183898
1,1,Sergi Darder,202648
2,2,Rafael,216547
3,3,Olivier Kemen,220175
4,4,Ivan Cavaleiro,212267
...,...,...,...
1818,1818,Iván Marcone,215057
1819,1819,Lucas Boyé,222041
1820,1820,Matthew Hoppe,259751
1821,1821,Dominik Greif,263139


In [7]:
df=no_match_df.copy()

In [8]:
df1=df.drop_duplicates(subset=['sofifa_id'])

In [9]:
df1

,Unnamed: 0,name,sofifa_id
0,0,Ángel Di María,183898
1,1,Sergi Darder,202648
2,2,Rafael,216547
3,3,Olivier Kemen,220175
4,4,Ivan Cavaleiro,212267
...,...,...,...
1818,1818,Iván Marcone,215057
1819,1819,Lucas Boyé,222041
1820,1820,Matthew Hoppe,259751
1821,1821,Dominik Greif,263139


In [10]:
df1.dtypes

Unnamed: 0     int64
name          object
sofifa_id     object
dtype: object

In [11]:
df1["name"]=df1["name"].astype("string")

/tmp/ipykernel_23173/1605082230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["name"]=df1["name"].astype("string")


In [12]:
df1=df1[df1["sofifa_id"]!="Nan"]

In [13]:
df1["sofifa_id"]=df1["sofifa_id"].astype("int64")

/tmp/ipykernel_23173/2171999497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["sofifa_id"]=df1["sofifa_id"].astype("int64")


In [14]:
df1.dtypes

Unnamed: 0     int64
name          string
sofifa_id      int64
dtype: object

#### Lower case names and remove accents

In [15]:
df1["name"]=df1["name"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df1.drop(df1.filter(regex="Unname"),axis=1, inplace=True)

/tmp/ipykernel_23173/3558419316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["name"]=df1["name"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
/home/ricardo/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [16]:
df1 = df1.rename(columns={'name': 'player_name'})

In [17]:
df1

,player_name,sofifa_id
0,angel di maria,183898
1,sergi darder,202648
2,rafael,216547
3,olivier kemen,220175
4,ivan cavaleiro,212267
...,...,...
1818,ivan marcone,215057
1819,lucas boye,222041
1820,matthew hoppe,259751
1821,dominik greif,263139


#### Preprocessing data from player transfer df . Lower casing, accent removing and cleaning data

In [18]:
import pandas as pd

player_transfer_df = pd.read_csv('/home/ricardo/Downloads/transfer_history_combined.csv') 
player_transfer_df["player_name"]=player_transfer_df["player_name"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
player_transfer_df.drop(player_transfer_df.filter(regex="Unname"),axis=1, inplace=True)
player_transfer_df["FIFA year"]=player_transfer_df["year"]

## dropping NaNs in fee_cleaned
player_transfer_df = player_transfer_df.loc[player_transfer_df["fee_cleaned"].notna()]
## dropping 0.0 transfer fees
player_transfer_df = player_transfer_df.loc[player_transfer_df["fee_cleaned"]!=0]
## dropping in vs. out duplications
player_transfer_df  = player_transfer_df.loc[player_transfer_df["transfer_movement"]=="in"]
player_transfer_df = player_transfer_df.loc[player_transfer_df["year"]>2014]

In [20]:
player_transfer_df

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,FIFA year
125629,Paris Saint-Germain,angel di maria,27.0,Right Winger,Man Utd,£56.70m,in,Summer,56.70,Ligue 1,2015,2015/2016,2015
125630,Paris Saint-Germain,layvin kurzawa,22.0,Left-Back,Monaco,£22.50m,in,Summer,22.50,Ligue 1,2015,2015/2016,2015
125631,Paris Saint-Germain,serge aurier,22.0,Right-Back,Toulouse,£9.00m,in,Summer,9.00,Ligue 1,2015,2015/2016,2015
125632,Paris Saint-Germain,kevin trapp,25.0,Goalkeeper,E. Frankfurt,£8.55m,in,Summer,8.55,Ligue 1,2015,2015/2016,2015
125633,Paris Saint-Germain,benjamin stambouli,24.0,Centre-Back,Spurs,£7.74m,in,Summer,7.74,Ligue 1,2015,2015/2016,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174297,RCD Mallorca,dominik greif,24.0,Goalkeeper,Slo. Bratislava,£2.25m,in,Summer,2.25,Primera Division,2021,2021/2022,2021
174298,RCD Mallorca,pablo maffeo,24.0,Right-Back,VfB Stuttgart,Loan fee:£450Th.,in,Summer,0.45,Primera Division,2021,2021/2022,2021
174330,Rayo Vallecano,fran garcia,22.0,Left-Back,RM Castilla,£1.80m,in,Summer,1.80,Primera Division,2021,2021/2022,2021
174331,Rayo Vallecano,randy nteka,23.0,Attacking Midfield,CF Fuenlabrada,£1.17m,in,Summer,1.17,Primera Division,2021,2021/2022,2021


#### Merge of the 2 tables above. ID scrapped players and the transfer dataset

In [21]:
player_transfer_sofifa_df = player_transfer_df.merge(df1, on=["player_name"], how='inner')

In [22]:
player_transfer_sofifa_df["player_name"].nunique()

1279

In [23]:
player_transfer_sofifa_df

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,FIFA year,sofifa_id
0,Paris Saint-Germain,angel di maria,27.0,Right Winger,Man Utd,£56.70m,in,Summer,56.70,Ligue 1,2015,2015/2016,2015,183898
1,Olympique Lyon,sergi darder,21.0,Central Midfield,Málaga CF,£11.79m,in,Summer,11.79,Ligue 1,2015,2015/2016,2015,202648
2,RCD Espanyol Barcelona,sergi darder,23.0,Central Midfield,Olympique Lyon,Loan fee:£270Th.,in,Summer,0.27,Primera Division,2017,2017/2018,2017,202648
3,RCD Espanyol Barcelona,sergi darder,24.0,Central Midfield,Olympique Lyon,£7.20m,in,Summer,7.20,Primera Division,2018,2018/2019,2018,202648
4,Olympique Lyon,rafael,25.0,Right-Back,Man Utd,£2.88m,in,Summer,2.88,Ligue 1,2015,2015/2016,2015,216547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,Elche CF,ivan marcone,31.0,Defensive Midfield,Boca Juniors,£3.60m,in,Summer,3.60,Primera Division,2021,2021/2022,2021,215057
1876,Elche CF,lucas boye,25.0,Centre-Forward,Torino,£1.80m,in,Summer,1.80,Primera Division,2021,2021/2022,2021,222041
1877,RCD Mallorca,matthew hoppe,20.0,Centre-Forward,FC Schalke 04,£2.70m,in,Summer,2.70,Primera Division,2021,2021/2022,2021,259751
1878,RCD Mallorca,dominik greif,24.0,Goalkeeper,Slo. Bratislava,£2.25m,in,Summer,2.25,Primera Division,2021,2021/2022,2021,263139


#### Importing data from player features df 

In [24]:
import pandas as pd

player_features_df = pd.read_csv('/home/ricardo/Downloads/players_combined.csv') 
player_features_df

/home/ricardo/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,FIFA year
0,0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,27,1987-06-24,169,67,Argentina,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,2015
1,1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,29,1985-02-05,185,80,Portugal,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,2015
2,2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,30,1984-01-23,180,80,Netherlands,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,2015
3,3,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,32,1981-10-03,195,95,Sweden,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,2015
4,4,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,28,1986-03-27,193,92,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100990,18273,245006,https://sofifa.com/player/245006/shuai-shao/20...,Shao Shuai,邵帅,22,1997-03-10,186,79,China PR,...,42+2,42+2,42+2,43+2,45+2,46+2,46+2,46+2,45+2,2020
100991,18274,250995,https://sofifa.com/player/250995/mingjie-xiao/...,Xiao Mingjie,Mingjie Xiao,22,1997-01-01,177,66,China PR,...,43+2,43+2,43+2,44+2,46+2,47+2,47+2,47+2,46+2,2020
100992,18275,252332,https://sofifa.com/player/252332/wei-zhang/20/...,Zhang Wei,张威,19,2000-05-16,186,75,China PR,...,49+2,49+2,49+2,47+2,47+2,49+2,49+2,49+2,47+2,2020
100993,18276,251110,https://sofifa.com/player/251110/haijian-wang/...,Wang Haijian,汪海健,18,2000-08-02,185,74,China PR,...,48+2,48+2,48+2,48+2,48+2,49+2,49+2,49+2,48+2,2020


#### Final merge between transfer and feature data on the sofifa_id

In [25]:
merged_df = player_transfer_sofifa_df.merge(player_features_df, on=["sofifa_id","FIFA year"], how='inner')

In [26]:
merged_df

,club_name,player_name,age_x,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,Paris Saint-Germain,angel di maria,27.0,Right Winger,Man Utd,£56.70m,in,Summer,56.70,Ligue 1,...,75+2,73+2,73+2,73+2,75+2,70+2,64+2,64+2,64+2,70+2
1,Olympique Lyon,sergi darder,21.0,Central Midfield,Málaga CF,£11.79m,in,Summer,11.79,Ligue 1,...,68+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,66+0,66+0
2,RCD Espanyol Barcelona,sergi darder,23.0,Central Midfield,Olympique Lyon,Loan fee:£270Th.,in,Summer,0.27,Primera Division,...,71+1,75+1,75+1,75+1,71+1,70+1,69+1,69+1,69+1,70+1
3,RCD Espanyol Barcelona,sergi darder,24.0,Central Midfield,Olympique Lyon,£7.20m,in,Summer,7.20,Primera Division,...,69+1,73+1,73+1,73+1,69+1,68+1,67+1,67+1,67+1,68+1
4,Olympique Lyon,rafael,25.0,Right-Back,Man Utd,£2.88m,in,Summer,2.88,Ligue 1,...,58+0,54+0,54+0,54+0,58+0,52+0,43+0,43+0,43+0,52+0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,Elche CF,raul guti,23.0,Central Midfield,Real Zaragoza,£4.50m,in,Summer,4.50,Primera Division,...,64+2,67+2,67+2,67+2,64+2,64+2,65+2,65+2,65+2,64+2
1366,Elche CF,tete morente,23.0,Left Winger,Málaga CF,£450Th.,in,Summer,0.45,Primera Division,...,55+2,54+2,54+2,54+2,55+2,53+2,50+2,50+2,50+2,53+2
1367,Real Sociedad,carlos fernandez,24.0,Centre-Forward,Sevilla FC,£9.00m,in,Winter,9.00,Primera Division,...,48+2,49+2,49+2,49+2,48+2,45+2,44+2,44+2,44+2,45+2
1368,Real Valladolid CF,lucas olaza,26.0,Left-Back,Boca Juniors,Loan fee:£450Th.,in,Winter,0.45,Primera Division,...,74+2,71+2,71+2,71+2,74+2,73+2,70+2,70+2,70+2,73+2


In [28]:
merged_df.player_name.unique()

array(['angel di maria', 'sergi darder', 'rafael', 'ivan cavaleiro',
       'rony lopes', 'guido carrillo', 'fabinho', 'lucas ocampos',
       'rolando', 'robert beric', 'pablo', 'casimir ninga',
       'serhou guirassy', 'juan fernando quintero',
       'kolbeinn sigthorsson', 'majeed waris', 'alexander soderlund',
       'mauro arambarri', 'arturo vidal', 'dante', 'francisco rodriguez',
       'matija nastasic', 'franco di santo', 'pierre-emile hojbjerg',
       'fabian schar', 'pavel kaderabek', 'joelinton', 'marko maric',
       'antonio colak', 'lukas hradecky', 'mijat gacinovic',
       'ulisses garcia', 'yoshinori muto', 'charlison benschop',
       'mitchell langerak', 'toni sunjic', 'przemyslaw tyton',
       'vladimir darida', 'lewis holtby', 'gotoku sakai', 'orjan nyland',
       'mario vrancic', 'papy djilobodji', 'raul jimenez', 'ederson',
       'giannelli imbula', 'jesus corona', 'danilo pereira',
       'andre andre', 'sergio oliveira', 'miguel layun', 'naldo',
       '

In [29]:
merged_df.player_name.nunique()

940

In [30]:
web_scrapped_final_table=merged_df

In [34]:
web_scrapped_final_table.to_csv("../raw_data/player_merged_by_fifaid.csv")